<h1>System of Linear Equations Routines</h1>

In [ ]:
# Let us first define the numbers we will work with. Note the use of KK when defining
# both matrices and vectors. Details aside it will make output numbers look nice if you set
# KK equal QQ. Other possible options are RR, CC, QQ, QQbar; here QQbar is another nice option
KK = QQ

def GetEch(M265Matrix):
    # given a matrix defined as above the function returns the echelon form of the matrix
    return M265Matrix.echelon_form()

def HomSol(M265Matrix):
    # obtain the homogeneous solution set of SLE with matrix M265Matrix
    return M265Matrix.right_kernel_matrix()

def ParSol(M265Matrix,M265Vector):
    # obtain a particular solutino to SLE with matrix M265Matrix and vector of constant M265Vector
    (rw,cl)=M265Matrix.dimensions()
    if(rw!=len(M265Vector)):
        print("dimension error")
    try:
        return M265Matrix.solve_right(M265Vector)
    except ValueError:
        #if ValueError occurs there is no solution but instead of raising the error we return
        # the empty set
        return vector(KK,[])
    return False
    
def GetERO(M265Matrix,debug=False):
    # get one list of elemetary operations that reduce a matrix to Reduced Echelon Form
    resEliminations = []
    [Rw,Cl]=M265Matrix.dimensions()
    M265copy = Matrix(KK,M265Matrix)
    if (debug):
        print(M265copy)
    rw = 0
    cl = 0
    for c in range(Cl):
        hasPivot = False
        # first we check if the column under question has a pivot
        for r in range(rw,Rw):
            if (M265copy[r][cl]!=0):
                hasPivot = True
                if (r!=rw):
                    EO = identity_matrix(KK,Rw)
                    EO.swap_rows(rw,r)
                    resEliminations.append(EO)
                    M265copy.swap_rows(rw,r)
                    if (debug):
                        print("\n")
                        print("Swap row ",rw,"with row", r)
                        print(M265copy)
                break # if there is a pivot 
        if (hasPivot):
            # if there is a pivot first scale it to one if the pivot is not one
            if (M265copy[rw][cl]!= KK(1)):
                EO=identity_matrix(KK,Rw)
                EO.rescale_row(rw,KK(1)/M265copy[rw][cl])
                resEliminations.append(EO)
                M265copy=resEliminations[-1]*M265copy
                if (debug):
                    print("\n")
                    print("Scale row", rw)
                    print(M265copy)

            # make the respective entries in the column zero except for the row that contains the pivot (r==rw)
            # further if a row has already entry zero at that column skip that row as the resulting scaling
            # is simply the identity matrix (M265copy[r][cl]==0)
            for r in range(Rw):
                if ((r==rw) or (M265copy[r][cl]==0)):
                    continue
                EO = identity_matrix(KK,Rw)
                EO.add_multiple_of_row(r,rw,-M265copy[r][cl])
                resEliminations.append(EO)
                #M265copy = M265copy.add_multiple_of_row(r,rw,-M265copy[r][cl])
                M265copy=resEliminations[-1]*M265copy
                if(debug):
                    print("\n")
                    print("M265copy multiple of row",rw,"to row",r )
                    print(M265copy)
            rw += 1 # if there was a pivot in this column for the remaining rows up we search for pivots
            # only on rows that are below it
        cl += 1
    return resEliminations

<h2>Some nice matrices</h2>

<h3>$3\times 3$ matrices</h3>

In [ ]:
M3 = Matrix(KK,[[5,-8,-4],[3,-5,-2],[-4,7,3]])
M4 = Matrix(KK, [\
                 [-3, -2, -8,  1],\
                 [-2,  0, -5,  1],\
                 [ 0, -1, -1,  0],\
                 [ 0, -5,  0, -2]])
M3, M4

<h1>Homework</h1>

<h2>Restrictions</h2>
<p>You may run the sage kernel or any other kernel of your choice. However, beyond the Standard Routines provided at the top of this notebook no other function calls are allowed from the sage (or any other library).</p>
<p>If you cannot run sage, please update the above routines with the solutions I've provided for the first homework and use those instead. You are welcome to contact me for further information if that is the case.</p>

<h2>Linear dependence</h2>
In the homework $M$ denotes a square matrix of order $d$ and  $\vec{v} = vv$ a vector with $d$ components that is different from the zero vector (i.e. $\vec{v}\ne \vec{0}$).

Consider the sequence $\vec{v}_0 = \vec{v}$, $\vec{v}_1 = M\vec{v}_0\dots$ defined recursively $$\vec{v}_{i+1} = M\vec{v}_i$$. Since all vectors lie in the same $d$-dimensional vector space then there is a smallest integer $k$ such that the set $\vec{v}_0,\dots\vec{v}_{k}$ is linearly dependent but any set  $\vec{v}_0,\dots\vec{v}_{j}$ with $j<k$ is linearly independent. With that in mind it is necessarily the case that in the equation $$\alpha_0\vec{v}_0 + \alpha_1\vec{v}_1+\cdots+\alpha_{k-1}\vec{v}_{k-1}+\alpha_{k}\vec{v}_k =\vec{0}$$ the coefficient $\alpha_k\ne 0$.

Your tasks are:
<dl>
    <dt>DepDegree(M,vv)</dt>
    <dd>Given a matrix $M$ and a vector $vv$ return the value $k$ described above.</dd> 
    <dt>DepCoefs(M,vv)</dt>
    <dd>Given a matrix $M$ and a vector $vv$ return the values $\frac{\alpha_0}{\alpha_k},\dots,\frac{\alpha_{k-1}}{\alpha_k}$ in a list.</dd>
</dl>
<h2>Remarks</h2>
When you program your routines your routines <em>must not</em> use Python's print function in <em>any</em> form. Your functions should simply return the above values.

In [ ]:
def DepDegree(MM, vv):
    res = 0
    vectors = [vv]
    coefficient_matrix = Matrix(QQ, vectors)
    
    while not HomSol(coefficient_matrix.transpose()): #coefficient_matrix.transpose().right_kernel_matrix():
        vectors.append(MM * vectors[-1])
        coefficient_matrix = Matrix(QQ, vectors)
        res += 1

    return res


def DepCoefs(MM, vv):
    count = 0
    vectors = [vv]
    coefficient_matrix = Matrix(QQ, vectors)
    
    while not HomSol(coefficient_matrix.transpose()): # coefficient_matrix.transpose().right_kernel_matrix():
        vectors.append(MM * vectors[-1])
        coefficient_matrix = Matrix(QQ, vectors)
        count += 1
        
    coefficient_matrix = Matrix(QQ, vectors[:count])
    constant_vector = vectors[count]
    
    print(coefficient_matrix.transpose().augment(constant_vector.column(), subdivide=True))
    
    return ParSol(coefficient_matrix.transpose(), constant_vector) # coefficient_matrix.transpose().solve_right(constant_vector)